### Quantum circuit learning to compute option prices and their sensitivites
<br>
A tutorial on using QCL to compute option prices and delta values.

In [1]:
import pennylane as qml
from pennylane import numpy as np
import scipy.stats as si
from matplotlib import pyplot as plt
import pandas as pd
import csv

random_seed = 0
np.random.seed(random_seed)

Suppose we have 7 training pairs ($S_i$, $V_i$) for $i = 1,2,...,7$. $S_i$ are stock prices and $V_i$ are European call option prices.

In [2]:
num_of_data = 7 
Spot = np.zeros(num_of_data) 

Spot[0] = 93
Spot[1] = 95
Spot[2] = 97
Spot[3] = 100
Spot[4] = 103
Spot[5] = 105
Spot[6] = 107

Given a stock price $S_i$, call option prices $V(S_i)$ are given by the Black-Scholes formula:
$$
    V(S_i)= S N(d_1) -  Ke^{-rt} N(d_2), 
$$    
$$
	d_1= \frac{1}{\sigma \sqrt{t}} [\ln{(\frac{S_i}{K})} + t(r + \frac{\sigma^2}{2})],
    d_2=d_1-\sigma \sqrt{t},
\\
\\
    N(x)=\frac{1}{\sqrt{2\pi}} \int_{-\infty}^{x} e^{-\frac{1}{2}z^2} dz.
$$
K is a strike price,t is time to maturity,r is risk-free rate and $\sigma$ is volatility.

In [3]:
V = np.zeros(num_of_data) 

K=100
T=0.25
r=0.0
sigma=0.15


def BSEuroCall(S, K, t, r, sigma):
        
    d1=(np.log(S/K)+(r+0.5*sigma**2)*t)/(sigma*np.sqrt(t))
    d2=d1-sigma*np.sqrt(t)
    
    price = S*si.norm.cdf(d1,0.0,1.0)-K*np.exp(-r*t)*si.norm.cdf(d2,0.0,1.0)
    
    return price

V = BSEuroCall(Spot, K, T, r,sigma)

In order to embed this real data into quantum state, first we convert $S_i$ and $V_i$ into x and y via
$$
    x(S_i)= tanh({(\frac{S_i}{C_1})}^\beta), y(V_i)= tanh({(\frac{V_i}{C_2})}^\gamma),
$$
where $C_1$, $C_2$, $\beta$ and $\gamma$ are fixed parameters.

In [4]:
beta=5.0
gamma=0.75
C1=K
C2=10 

X=np.tanh(np.power(Spot/C1,beta))       
Y=np.tanh(np.power(V/C2,gamma)) 

Then we apply the following $U_{in}$ to the initial state $|0\rangle$:
$$
|\Psi_{in}(\varphi_1,...,\varphi_5)\rangle = U_{in}(\varphi_1,...,\varphi_5)|0\rangle,
$$
where 
$$
U_{in}(\varphi_1,...,\varphi_5)=[R_2^{Y}(\varphi_5) \otimes R_1^{Y}(\varphi_4)] R^{XX}(\varphi_3) [R_2^{Y}(\varphi_2) \otimes R_1^{Y}(\varphi_1)]
\\
\\
$$
and $\varphi_1(x)=\varphi_2(x)=...\varphi_5(x)=arcsin(x)$.  
Appyling a parameterized quantum circuit $U(\theta)$ gives the final state as
$$
  |\Psi_{out}(\theta,\varphi_1,...,\varphi_5)\rangle=U(\theta)|\Psi^{in}(\varphi_1,...,\varphi_5)\rangle,
$$
where a pennylane template, StronglyEntanglingLayer is chosed as $U(\theta)$ with the number of layers 3 and the number of qubits 2 .
Finally measuring some observable $B$ gives 
$$
  \langle B(\theta,\varphi_1,...,\varphi_5)\rangle \equiv \langle\Psi_{out}|B|\Psi_{out}\rangle
$$
and we regard this as output $y(S_i,\theta,\varphi_1,...,\varphi_5)$.

In [5]:
n_qubits = 2 
n_layers = 3 
dev = qml.device("default.qubit", wires=n_qubits) 

var_init = np.random.uniform(high=2 * np.pi, size=(n_layers, n_qubits, 3))

@qml.qnode(dev, diff_method='parameter-shift')
def quantum_neural_net(var, x):
    
    angle_phi1 = np.arcsin(x) 
    angle_phi2 = np.arcsin(x)
    qml.RY(angle_phi1, wires=0)
    qml.RY(angle_phi2, wires=1)
     
    angle_phi3 = np.arcsin(x)  
    
    qml.IsingXX(angle_phi3, wires=[0, 1])  
         
    angle_phi4 = np.arcsin(x) 
    angle_phi5 = np.arcsin(x)
    qml.RY(angle_phi4, wires=0)  
    qml.RY(angle_phi5, wires=1)  
    
    qml.templates.StronglyEntanglingLayers(var, wires=range(n_qubits))
    
    return qml.expval(qml.PauliZ(0))

To compute delta value $\frac{\partial V(S)}{\partial S}$ via parameter-shift rules
we need
$$
\langle B(\theta,\varphi_1+\frac{\pi}{2},\varphi_2,\varphi_3,\varphi_4,\varphi_5)\rangle
$$
$$
\langle B(\theta,\varphi_1,\varphi_2+\frac{\pi}{2},\varphi_3,\varphi_4,\varphi_5)\rangle
$$
$$
\langle B(\theta,\varphi_1,\varphi_2,\varphi_3+\frac{\pi}{2},\varphi_4,\varphi_5)\rangle
$$
$$
\langle B(\theta,\varphi_1,\varphi_2,\varphi_3,\varphi_4+\frac{\pi}{2},\varphi_5)\rangle
$$
$$
\langle B(\theta,\varphi_1,\varphi_2,\varphi_3,\varphi_4,\varphi_5+\frac{\pi}{2})\rangle
$$
and
$$
\langle B(\theta,\varphi_1-\frac{\pi}{2},\varphi_2,\varphi_3,\varphi_4,\varphi_5)\rangle
$$
$$
\langle B(\theta,\varphi_1,\varphi_2-\frac{\pi}{2},\varphi_3,\varphi_4,\varphi_5)\rangle
$$
$$
\langle B(\theta,\varphi_1,\varphi_2,\varphi_3-\frac{\pi}{2},\varphi_4,\varphi_5)\rangle
$$
$$
\langle B(\theta,\varphi_1,\varphi_2,\varphi_3,\varphi_4-\frac{\pi}{2},\varphi_5)\rangle
$$
$$
\langle B(\theta,\varphi_1,\varphi_2,\varphi_3,\varphi_4,\varphi_5-\frac{\pi}{2})\rangle.
$$

In [6]:
@qml.qnode(dev, diff_method='parameter-shift') 
def quantum_neural_net_1p(var, x):
    
    angle_phi1 = np.arcsin(x)+np.pi*0.5  
    angle_phi2 = np.arcsin(x)
    qml.RY(angle_phi1, wires=0)
    qml.RY(angle_phi2, wires=1)
     
    angle_phi3 = np.arcsin(x)  
    
    qml.IsingXX(angle_phi3, wires=[0, 1])  
         
    angle_phi4 = np.arcsin(x) 
    angle_phi5 = np.arcsin(x)
    qml.RY(angle_phi4, wires=0)  
    qml.RY(angle_phi5, wires=1)  
    
    qml.templates.StronglyEntanglingLayers(var, wires=range(n_qubits))
    
    return qml.expval(qml.PauliZ(0))

@qml.qnode(dev, diff_method='parameter-shift') 
def quantum_neural_net_1m(var, x):
    
    angle_phi1 = np.arcsin(x)-np.pi*0.5  
    angle_phi2 = np.arcsin(x)
    qml.RY(angle_phi1, wires=0)
    qml.RY(angle_phi2, wires=1)
     
    angle_phi3 = np.arcsin(x)  
    
    qml.IsingXX(angle_phi3, wires=[0, 1])  
         
    angle_phi4 = np.arcsin(x) 
    angle_phi5 = np.arcsin(x)
    qml.RY(angle_phi4, wires=0)  
    qml.RY(angle_phi5, wires=1)  
    
    qml.templates.StronglyEntanglingLayers(var, wires=range(n_qubits))
    
    return qml.expval(qml.PauliZ(0))

@qml.qnode(dev, diff_method='parameter-shift') 
def quantum_neural_net_2p(var, x):
    
    angle_phi1 = np.arcsin(x)
    angle_phi2 = np.arcsin(x)+np.pi*0.5  
    qml.RY(angle_phi1, wires=0)
    qml.RY(angle_phi2, wires=1)
     
    angle_phi3 = np.arcsin(x)  
    
    qml.IsingXX(angle_phi3, wires=[0, 1])  
         
    angle_phi4 = np.arcsin(x) 
    angle_phi5 = np.arcsin(x)
    qml.RY(angle_phi4, wires=0)  
    qml.RY(angle_phi5, wires=1)  
    
    qml.templates.StronglyEntanglingLayers(var, wires=range(n_qubits))
    
    return qml.expval(qml.PauliZ(0))

@qml.qnode(dev, diff_method='parameter-shift') 
def quantum_neural_net_2m(var, x):
    
    angle_phi1 = np.arcsin(x)
    angle_phi2 = np.arcsin(x)-np.pi*0.5  
    qml.RY(angle_phi1, wires=0)
    qml.RY(angle_phi2, wires=1)
     
    angle_phi3 = np.arcsin(x)  
    
    qml.IsingXX(angle_phi3, wires=[0, 1])  
         
    angle_phi4 = np.arcsin(x) 
    angle_phi5 = np.arcsin(x)
    qml.RY(angle_phi4, wires=0)  
    qml.RY(angle_phi5, wires=1)  
     
    qml.templates.StronglyEntanglingLayers(var, wires=range(n_qubits))
    
    return qml.expval(qml.PauliZ(0))

@qml.qnode(dev, diff_method='parameter-shift') 
def quantum_neural_net_3p(var, x):
    
    angle_phi1 = np.arcsin(x)
    angle_phi2 = np.arcsin(x)  
    qml.RY(angle_phi1, wires=0)
    qml.RY(angle_phi2, wires=1)
     
    angle_phi3 = np.arcsin(x)+np.pi*0.5  
    
    qml.IsingXX(angle_phi3, wires=[0, 1])  
         
    angle_phi4 = np.arcsin(x) 
    angle_phi5 = np.arcsin(x)
    qml.RY(angle_phi4, wires=0)  
    qml.RY(angle_phi5, wires=1)  
    
    qml.templates.StronglyEntanglingLayers(var, wires=range(n_qubits))
    
    return qml.expval(qml.PauliZ(0))

@qml.qnode(dev, diff_method='parameter-shift') 
def quantum_neural_net_3m(var, x):
    
    angle_phi1 = np.arcsin(x)
    angle_phi2 = np.arcsin(x)  
    qml.RY(angle_phi1, wires=0)
    qml.RY(angle_phi2, wires=1)
     
    angle_phi3 = np.arcsin(x)-np.pi*0.5  
    
    qml.IsingXX(angle_phi3, wires=[0, 1])  
         
    angle_phi4 = np.arcsin(x) 
    angle_phi5 = np.arcsin(x)
    qml.RY(angle_phi4, wires=0)  
    qml.RY(angle_phi5, wires=1)  
    
    qml.templates.StronglyEntanglingLayers(var, wires=range(n_qubits))
    
    return qml.expval(qml.PauliZ(0))

@qml.qnode(dev, diff_method='parameter-shift') 
def quantum_neural_net_4p(var, x):
    
    angle_phi1 = np.arcsin(x)
    angle_phi2 = np.arcsin(x)  
    qml.RY(angle_phi1, wires=0)
    qml.RY(angle_phi2, wires=1)
     
    angle_phi3 = np.arcsin(x) 
    
    qml.IsingXX(angle_phi3, wires=[0, 1])  
         
    angle_phi4 = np.arcsin(x)+np.pi*0.5 
    angle_phi5 = np.arcsin(x)
    qml.RY(angle_phi4, wires=0)  
    qml.RY(angle_phi5, wires=1)  
    
    qml.templates.StronglyEntanglingLayers(var, wires=range(n_qubits))
    
    return qml.expval(qml.PauliZ(0))

@qml.qnode(dev, diff_method='parameter-shift') 
def quantum_neural_net_4m(var, x):
    
    angle_phi1 = np.arcsin(x)
    angle_phi2 = np.arcsin(x)  
    qml.RY(angle_phi1, wires=0)
    qml.RY(angle_phi2, wires=1)
     
    angle_phi3 = np.arcsin(x) 
    
    qml.IsingXX(angle_phi3, wires=[0, 1])  
         
    angle_phi4 = np.arcsin(x)-np.pi*0.5 
    angle_phi5 = np.arcsin(x)
    qml.RY(angle_phi4, wires=0)  
    qml.RY(angle_phi5, wires=1)  
    
    qml.templates.StronglyEntanglingLayers(var, wires=range(n_qubits))
    
    return qml.expval(qml.PauliZ(0))

 
@qml.qnode(dev, diff_method='parameter-shift')
def quantum_neural_net_5p(var, x):
    
    angle_phi1 = np.arcsin(x)
    angle_phi2 = np.arcsin(x)  
    qml.RY(angle_phi1, wires=0)
    qml.RY(angle_phi2, wires=1)
     
    angle_phi3 = np.arcsin(x) 
    
    qml.IsingXX(angle_phi3, wires=[0, 1])  
         
    angle_phi4 = np.arcsin(x)
    angle_phi5 = np.arcsin(x)+np.pi*0.5 
    qml.RY(angle_phi4, wires=0)  
    qml.RY(angle_phi5, wires=1)  
    
    qml.templates.StronglyEntanglingLayers(var, wires=range(n_qubits))
    
    return qml.expval(qml.PauliZ(0))


@qml.qnode(dev, diff_method='parameter-shift') 
def quantum_neural_net_5m(var, x):
    
    angle_phi1 = np.arcsin(x)
    angle_phi2 = np.arcsin(x)  
    qml.RY(angle_phi1, wires=0)
    qml.RY(angle_phi2, wires=1)
     
    angle_phi3 = np.arcsin(x) 
    
    qml.IsingXX(angle_phi3, wires=[0, 1])  
         
    angle_phi4 = np.arcsin(x)
    angle_phi5 = np.arcsin(x)-np.pi*0.5 
    qml.RY(angle_phi4, wires=0)  
    qml.RY(angle_phi5, wires=1)  
    
    qml.templates.StronglyEntanglingLayers(var, wires=range(n_qubits))
    
    return qml.expval(qml.PauliZ(0))

Loss functions is given as
$$
loss= \frac{1}{7} \sum_{i=1}^{7}\frac{(y(V_i)-y(S_i,\theta,\varphi_1,...,\varphi_5))^2}{y(V_i)}.
$$

In [7]:
def square_loss(desired, predictions):
    loss = 0
    for l, p in zip(desired, predictions):
        loss = loss + ((l - p) ** 2)/l 
    loss = loss / len(desired)
    return loss

def cost(var, features1,desired):
    preds = [quantum_neural_net(var, x) for x in features1]
   
    return square_loss(desired, preds)

Adam optimizer is used with a learning rate of 0.01 for 1000 iterations. The gradients in the Adam optimizer
are computed analytically by parameter-shift rules.

In [8]:
opt = qml.AdamOptimizer(0.01) 

hist_cost = []
var = var_init
for it in range(1000): 
    var, _cost = opt.step_and_cost(lambda v: cost(v,X,Y), var)
    if it % 50 == 0:
        print("Iter:"+str(it)+", cost="+str(_cost.numpy()))
    hist_cost.append(_cost)

Iter:0, cost=0.5958833133945671
Iter:50, cost=0.0028332607855504933
Iter:100, cost=0.0004960147282667399
Iter:150, cost=0.00046330687227837824
Iter:200, cost=0.00043056954363887145
Iter:250, cost=0.0003898403429854614
Iter:300, cost=0.0003411937340694918
Iter:350, cost=0.00028566865504111444
Iter:400, cost=0.00022564231299293316
Iter:450, cost=0.00016509791707670292
Iter:500, cost=0.00010930435548359433
Iter:550, cost=6.35673704260778e-05
Iter:600, cost=3.1280565156937564e-05
Iter:650, cost=1.2410626909849379e-05
Iter:700, cost=3.729744252190445e-06
Iter:750, cost=8.029809370684376e-07
Iter:800, cost=1.5483994671390644e-07
Iter:850, cost=7.556238525097817e-08
Iter:900, cost=7.075510686197105e-08
Iter:950, cost=6.934072266591735e-08


Prices learned by the quantum circuit are given by
$V_{QCL}=C_2 (arctanh(y(S_i,\theta,\varphi))^{\frac{1}{\gamma}} $.

In [9]:
print('\n training price')
for i in range(num_of_data):
    print(V[i])  

Y_pred = [quantum_neural_net(var, x) for x in X]
V_QCL=np.power(np.arctanh(Y_pred),1/gamma)*C2
    
print('\n learned price')
for i in range(num_of_data):
    print(V_QCL[i]) 


 training price
0.6404129465494446
1.0727938394938548
1.6860656488366885
2.9913659851819503
4.768907714177985
6.1925459658196615
7.775800095424174

 learned price
0.6400488021468097
1.0731720594494631
1.6876924546962935
2.9899659800202567
4.764308465590926
6.192682218815452
7.781837954997643


Training delta can be computed anaytically using the Black-Scholes formula:
$$
\frac{\partial V(S)}{\partial S}=N(d_1).
$$

In [10]:
def BSEuroCallDelta(S, K, t, r, sigma):
    
    d1=(np.log(S/K)+(r+0.5*sigma**2)*t)/(sigma*np.sqrt(t))
    
    delta = si.norm.cdf(d1,0.0,1.0)
    
    return delta

V_delta = BSEuroCallDelta(Spot, K, T, r,sigma)

print('\n training delta')
for i in range(num_of_data):
    print(V_delta[i]) 


 training delta
0.17615726414092164
0.25900674135623625
0.3562044618273095
0.5149568299259097
0.6669902316392698
0.7542847939040045
0.8262925406465411


We can compute delta values from learned quatum circuit via parameter-shift rules:
$$
\frac{\partial V}{\partial S}=\frac{\partial \langle B(\theta,\varphi_1,...,\varphi_5)\rangle }{\partial S}\frac{\partial y}{\partial V},
$$
where 
$$
\frac{\partial \langle B(\theta,\varphi_1,...,\varphi_5)\rangle }{\partial S}=\frac{\partial x}{\partial S}\sum_{j=1}^{5}\frac{\partial \varphi_j}{\partial x}\frac{\partial \langle B(\theta,\varphi_1,...,\varphi_5)\rangle }{\partial \varphi_j},
\\
\frac{\partial \varphi_j}{\partial x}=\frac{1}{\sqrt{1-x^2}},
\frac{\partial x}{\partial S}=\frac{\beta}{C_1}(\frac{x}{C_1})^{\beta-1}(1-x^2),
\frac{\partial y}{\partial V}=\frac{\gamma}{C_2}(\frac{V}{C_2})^{\gamma-1}(1-y^2),
\\
\frac{\partial \langle B(\theta,\varphi_1,...,\varphi_5)\rangle }{\partial \varphi_j}=\frac{1}{2}(\langle B(\theta,...,\varphi_j+\frac{\pi}{2},...)\rangle-\langle B(\theta,...,\varphi_j-\frac{\pi}{2},...)\rangle).
$$


In [11]:
xd=np.array([beta/C1*np.power(S/C1,beta-1)*(1-np.power(x,2))for S,x in zip(Spot, X)])
yd_pred=np.array([gamma/C2*np.power(V_l/C2,gamma-1)*(1-np.power(yd,2))for V_l,yd in zip(V_QCL,Y_pred)])

Yd_1p = np.array([xd_t/yd_p*0.5*1/np.power(1-np.power(x,2),0.5)*quantum_neural_net_1p(var, x) for x,xd_t,yd_p in zip(X,xd,yd_pred)])
Yd_1m = np.array([xd_t/yd_p*0.5*1/np.power(1-np.power(x,2),0.5)*quantum_neural_net_1m(var, x) for x,xd_t,yd_p in zip(X,xd,yd_pred)])

Yd_2p = np.array([xd_t/yd_p*0.5*1/np.power(1-np.power(x,2),0.5)*quantum_neural_net_2p(var, x) for x,xd_t,yd_p in zip(X,xd,yd_pred)])
Yd_2m = np.array([xd_t/yd_p*0.5*1/np.power(1-np.power(x,2),0.5)*quantum_neural_net_2m(var, x) for x,xd_t,yd_p in zip(X,xd,yd_pred)])

Yd_3p = np.array([xd_t/yd_p*0.5*1/np.power(1-np.power(x,2),0.5)*quantum_neural_net_3p(var, x) for x,xd_t,yd_p in zip(X,xd,yd_pred)])
Yd_3m = np.array([xd_t/yd_p*0.5*1/np.power(1-np.power(x,2),0.5)*quantum_neural_net_3m(var, x) for x,xd_t,yd_p in zip(X,xd,yd_pred)])

Yd_4p = np.array([xd_t/yd_p*0.5*1/np.power(1-np.power(x,2),0.5)*quantum_neural_net_4p(var, x) for x,xd_t,yd_p in zip(X,xd,yd_pred)])
Yd_4m = np.array([xd_t/yd_p*0.5*1/np.power(1-np.power(x,2),0.5)*quantum_neural_net_4m(var, x) for x,xd_t,yd_p in zip(X,xd,yd_pred)])

Yd_5p = np.array([xd_t/yd_p*0.5*1/np.power(1-np.power(x,2),0.5)*quantum_neural_net_5p(var, x) for x,xd_t,yd_p in zip(X,xd,yd_pred)])
Yd_5m = np.array([xd_t/yd_p*0.5*1/np.power(1-np.power(x,2),0.5)*quantum_neural_net_5m(var, x) for x,xd_t,yd_p in zip(X,xd,yd_pred)])


V_delta_learned=Yd_1p-Yd_1m+Yd_2p-Yd_2m+Yd_3p-Yd_3m+Yd_4p-Yd_4m+Yd_5p-Yd_5m
    
print('\n learned delta')
for i in range(num_of_data):
    print(V_delta_learned[i]) 


 learned delta
0.17568385330145408
0.259856484085488
0.35632006168815406
0.5131162936933921
0.6676809362832505
0.7580248133448244
0.8263631174756644


#### References
[1] Sakuma,T. Quantum Circuit Learning to Compute Option Prices and Their Sensitivities (September 13, 2021). Available at SSRN: https://ssrn.com/abstract=3922040
        
